In [1]:
import numpy as np
import os
import sys
import torch
import pickle
import re
from collections import OrderedDict
from networks_2022 import BranchedNetwork

In [2]:
# process raw weights 

In [4]:
import pickle

In [5]:
weights_path = './weights/'
with open(weights_path+ 'net7_tf_2020w_naturalistic_not_resampled_held_out_speaker_NOTIMIT_seed_0_1en4.0.0001.sum_0.0001_ckpt_7conv5_W_py3.cpy', 'rb') as f:
    weights_biases = pickle.load(f,  encoding="latin1")


In [6]:
files = os.listdir(weights_path)

weights_biases = {}
for f in files:
    if 'py3' in f:
        
        layer = f.split('7')[2].split('.cpy')[0].split('_py3')[0]
        print(layer)
        with open(weights_path+ f, 'rb') as f_:
            wb = pickle.load(f_,  encoding="latin1")
            weights_biases[layer] = wb

conv5_G
fctop_W
conv4_G
fc6_W
fc6_G
conv3
conv4_W
conv2
fctop_G
conv5_W
conv1


In [7]:
word_branch = weights_biases

In [8]:
genre_branch= weights_biases

In [9]:
bn = BranchedNetwork()
keys = bn.state_dict().keys()

/Users/chingfang/Code/hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/Users/chingfang/Code/hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


In [10]:
regex = re.compile('(\w+_branch)\.(\w+)\.block\.(?:0\.)*(\w+)')

In [11]:
state_dict = OrderedDict()
for key in keys:
    print(key)
    key_groups = regex.match(key)
    branch = key_groups.group(1)
    layer = key_groups.group(2)
    struct = key_groups.group(3)
    struct = 'W' if 'weight' in struct else 'b'
    if layer.startswith('fc') and layer.endswith('W'):
        arr = word_branch[layer][struct]
    elif layer.startswith('fc') and layer.endswith('G'):
        arr = genre_branch[layer][struct]
    else:
        arr = weights_biases[layer][struct]
    arr = torch.tensor(arr)
    if len(arr.shape) == 4:
        arr = torch.permute(arr, (3, 2, 0, 1))
    elif len(arr.shape) == 2:
        arr = torch.permute(arr, (1, 0))
    state_dict[key] = arr
    print(np.shape(arr))

speech_branch.conv1.block.0.weight
torch.Size([96, 1, 6, 14])
speech_branch.conv1.block.0.bias
torch.Size([96])
speech_branch.conv2.block.0.weight
torch.Size([256, 96, 5, 5])
speech_branch.conv2.block.0.bias
torch.Size([256])
speech_branch.conv3.block.0.weight
torch.Size([512, 256, 3, 3])
speech_branch.conv3.block.0.bias
torch.Size([512])
speech_branch.conv4_W.block.0.weight
torch.Size([1024, 512, 3, 3])
speech_branch.conv4_W.block.0.bias
torch.Size([1024])
speech_branch.conv5_W.block.0.weight
torch.Size([512, 1024, 3, 3])
speech_branch.conv5_W.block.0.bias
torch.Size([512])
speech_branch.fc6_W.block.0.weight
torch.Size([4096, 18432])
speech_branch.fc6_W.block.0.bias
torch.Size([4096])
speech_branch.fctop_W.block.weight
torch.Size([531, 4096])
speech_branch.fctop_W.block.bias
torch.Size([531])
genre_branch.conv1.block.0.weight
torch.Size([96, 1, 6, 14])
genre_branch.conv1.block.0.bias
torch.Size([96])
genre_branch.conv2.block.0.weight
torch.Size([256, 96, 5, 5])
genre_branch.conv2.bloc

In [13]:
torch.save(state_dict, './weights/networks_2022_weights.pt')